##################################### DISEASE PREDICTION #######################################

This is a temporary script file.

dataset - adm+patients
dataset1 - dataset+diag(iacd id as columns, encoded)
df_lab_evnts - dataset+lab_evnts(valnum according to hadm id)



In [1]:
#import dask
import pandas as pd
import datetime as dt
from datetime import date,time,datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer
lbl = LabelEncoder()
mlb = MultiLabelBinarizer()
#import dask.dataframe as dd
import numpy as np
import pickle as p

In [2]:
adm = pd.read_csv('ADMISSIONS.csv',parse_dates=[3,4],date_parser=lambda x:datetime.strptime(x,"%Y-%m-%d %H:%M:%S"))
icu_stays = pd.read_csv('ICUSTAYS.csv')
patients = pd.read_csv('PATIENTS.csv',parse_dates=[3],date_parser=lambda x:datetime.strptime(x,"%Y-%m-%d %H:%M:%S"))
diag_ref = pd.read_csv('D_ICD_DIAGNOSES.csv')
#lab_itms = pd.read_csv('D_LABITEMS.csv')
diag = pd.read_csv('DIAGNOSES_ICD.csv')
lab_evnts = pd.read_csv('LABEVENTS.csv')

In [3]:
patients.head()

,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
0,234,249,F,2075-03-13,NaN,NaN,NaN,0
1,235,250,F,2164-12-27,2188-11-22 00:00:00,2188-11-22 00:00:00,NaN,1
2,236,251,M,2090-03-15,NaN,NaN,NaN,0
3,237,252,M,2078-03-06,NaN,NaN,NaN,0
4,238,253,F,2089-11-26,NaN,NaN,NaN,0


In [4]:
lab_evnts = lab_evnts.dropna(subset = ['HADM_ID'], axis=0)

#doubtful on edregtime,edouttime
adm = adm.drop(['DEATHTIME','ADMISSION_LOCATION','DISCHARGE_LOCATION','INSURANCE','LANGUAGE',
                'MARITAL_STATUS','ETHNICITY','HOSPITAL_EXPIRE_FLAG','RELIGION','ROW_ID',
                'HAS_CHARTEVENTS_DATA','ADMISSION_TYPE','EDREGTIME','EDOUTTIME','DIAGNOSIS'],axis=1)

# Dropping some unnecessary columns from patients
patients = patients.drop(['ROW_ID','DOD','DOD_HOSP','DOD_SSN','EXPIRE_FLAG'],axis=1)

patients['GENDER'] = lbl.fit_transform(patients['GENDER'])

diag = diag.drop(['ROW_ID','SEQ_NUM','SUBJECT_ID'],axis = 1)


In [5]:
'''
#not sure about admission_type column,could lead to data leakage...
lbl = LabelEncoder()
adm.iloc[:,[5]] = lbl.fit_transform(adm.iloc[:,[5]])
#EMERGENCY-1
#ELECTIVE-0
#URGENT-3
#NEWBORN-2'''


adm['LENGTH_OF_STAY'] = abs(adm['ADMITTIME']-adm['DISCHTIME'])
days = pd.to_datetime(adm['LENGTH_OF_STAY']).dt.day
adm['ADMISSION_DAYS'] = days

# Merging 
dataset = pd.merge(adm,patients,how='left',left_on='SUBJECT_ID',right_on='SUBJECT_ID')
dataset['ADMITTIME'] = dataset['ADMITTIME'].dt.year
dataset['DOB'] = dataset['DOB'].dt.year
dataset['AGE'] = (dataset['ADMITTIME']-dataset['DOB']) 

age = []
for x in dataset['AGE']:
    if x>=300:
        x=x-300+89
        age.append(x)
    else:
        age.append(x)
dataset['AGE'] = age
    
dataset = dataset.drop(['ADMITTIME','DISCHTIME','LENGTH_OF_STAY','DOB'],axis=1)
adm = adm.drop(['ADMITTIME','DISCHTIME','LENGTH_OF_STAY'],axis=1)
diag['ICD9_CODE'] = diag.ICD9_CODE.astype(str)

/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: Passing timedelta64-dtype data is deprecated, will raise a TypeError in a future version
  if sys.path[0] == '':


#### Adding lab tests as columns 

In [6]:
#patients = patients.drop(['DOB'],axis=1)

lab_evnts = lab_evnts.drop(['VALUE','FLAG','VALUEUOM','ROW_ID'],axis=1)

# dropping rows with no HADM_ID, Values
lab_evnts = lab_evnts.dropna(subset=['HADM_ID'],axis=0)
lab_evnts = lab_evnts.dropna(subset=['VALUENUM'],axis=0)

In [7]:
# as of now retaining the value num of the first test occurence during a particular admission

lab_evnts = lab_evnts.sort_values('CHARTTIME').groupby(['HADM_ID','ITEMID']).head(1)
lab_evnts = lab_evnts.drop(['CHARTTIME'],axis=1)

# splitting to columns 
lab1 = lab_evnts.pivot_table('VALUENUM', ['HADM_ID'], 'ITEMID')

In [8]:
lab1.columns

Int64Index([50801, 50802, 50803, 50804, 50805, 50806, 50808, 50809, 50810,
            50811,
            ...
            51512, 51513, 51514, 51515, 51516, 51517, 51518, 51519, 51529,
            51532],
           dtype='int64', name='ITEMID', length=480)

In [9]:

dataset = pd.merge(dataset,lab1,how='left',left_on='HADM_ID',right_index=True)


#### Adding ICD codes as columns 

In [10]:

diag = pd.pivot_table(diag,index='HADM_ID',values='ICD9_CODE',aggfunc=list)
dataset = pd.merge(dataset,diag,how='outer',left_on='HADM_ID',right_index=True)
dataset1 = pd.DataFrame(mlb.fit_transform(dataset['ICD9_CODE']),columns=mlb.classes_, index=dataset.HADM_ID)
dataset1 = dataset1.drop('nan',axis=1)
dataset = dataset.drop('ICD9_CODE',axis=1)

**Grouping ICD Codes**

In [11]:
g1 = [x for x in dataset1.columns if x.isdigit() and int(x.lstrip('0')[0:3]) in range(1,140)]
g2=[x for x in dataset1.columns if x.isdigit() and int(x.lstrip('0')[0:3]) in range(140,240)]
g3=[x for x in dataset1.columns if x.isdigit() and int(x.lstrip('0')[0:3]) in range(240,280)]
g4=[x for x in dataset1.columns if x.isdigit() and int(x.lstrip('0')[0:3]) in range(280,290)]
g5=[x for x in dataset1.columns if x.isdigit() and int(x.lstrip('0')[0:3]) in range(290,320)]
g6=[x for x in dataset1.columns if x.isdigit() and int(x.lstrip('0')[0:3]) in range(320,390)]
g7=[x for x in dataset1.columns if x.isdigit() and int(x.lstrip('0')[0:3]) in range(390,460)]
g8=[x for x in dataset1.columns if x.isdigit() and int(x.lstrip('0')[0:3]) in range(460,520)]
g9=[x for x in dataset1.columns if x.isdigit() and int(x.lstrip('0')[0:3]) in range(520,580)]
g10=[x for x in dataset1.columns if x.isdigit() and int(x.lstrip('0')[0:3]) in range(580,630)]
g11=[x for x in dataset1.columns if x.isdigit() and int(x.lstrip('0')[0:3]) in range(630,678)]
g12=[x for x in dataset1.columns if x.isdigit() and int(x.lstrip('0')[0:3]) in range(680,710)]
g13=[x for x in dataset1.columns if x.isdigit() and int(x.lstrip('0')[0:3]) in range(710,740)]
g14=[x for x in dataset1.columns if x.isdigit() and int(x.lstrip('0')[0:3]) in range(740,760)]
g15=[x for x in dataset1.columns if x.isdigit() and int(x.lstrip('0')[0:3]) in range(760,780)]
g16=[x for x in dataset1.columns if x.isdigit() and int(x.lstrip('0')[0:3]) in range(780,797)]
g17=[x for x in dataset1.columns if x.isdigit() and int(x.lstrip('0')[0:3]) in range(797,800)]
g18=[x for x in dataset1.columns if x.isdigit() and int(x.lstrip('0')[0:3]) in range(800,1000)]
g19=[x for x in dataset1.columns if x.isalnum() and 'E' in x]
g20=[x for x in dataset1.columns if x.isalnum() and 'V' in x]

In [12]:
print(len(g1),len(g2),len(g3),len(g4),len(g5),len(g6),len(g7),len(g8),len(g9),len(g10),len(g11),len(g12),len(g13),len(g14),len(g15),len(g16),len(g17),len(g18),len(g19),len(g20))

102 502 251 108 257 565 452 206 406 260 203 159 374 262 206 264 18 1396 502 491


In [13]:
dataset1['G1']=dataset1[g1].max(1)
dataset1['G2']=dataset1[g2].max(1)
dataset1['G3']=dataset1[g3].max(1)
dataset1['G4']=dataset1[g4].max(1)
dataset1['G5']=dataset1[g5].max(1)
dataset1['G6']=dataset1[g6].max(1)
dataset1['G7']=dataset1[g7].max(1)
dataset1['G8']=dataset1[g8].max(1)
dataset1['G9']=dataset1[g9].max(1)
dataset1['G10']=dataset1[g10].max(1)
dataset1['G11']=dataset1[g11].max(1)
dataset1['G12']=dataset1[g12].max(1)
dataset1['G13']=dataset1[g13].max(1)
dataset1['G14']=dataset1[g14].max(1)
dataset1['G15']=dataset1[g15].max(1)
dataset1['G16']=dataset1[g16].max(1)
dataset1['G17']=dataset1[g17].max(1)
dataset1['G18']=dataset1[g18].max(1)
dataset1['G19']=dataset1[g19].max(1)
dataset1['G20']=dataset1[g20].max(1)

In [14]:
len(g1+g2+g3+g4+g5+g6+g7+g8+g9+g10+g11+g12+g13+g14+g15+g16+g17+g18+g19+g20)

6984

In [15]:
dataset1=dataset1.drop(g1+g2+g3+g4+g5+g6+g7+g8+g9+g10+g11+g12+g13+g14+g15+g16+g17+g18+g19+g20,1)

In [16]:
dataset1.shape

(58976, 20)

In [17]:
dataset1

,G1,G2,G3,G4,G5,G6,G7,G8,G9,G10,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20
HADM_ID,,,,,,,,,,,,,,,,,,,,
165315,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0
152223,0,0,1,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,1
124321,0,1,1,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,1
161859,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0
129635,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191113,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0
101071,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
122631,0,1,1,0,0,0,0,1,1,1,0,1,0,1,0,0,0,1,1,1


In [18]:
# final merge 
dataset = pd.merge(dataset,dataset1,how='left',left_on='HADM_ID',right_index=True)

In [19]:
dataset

,SUBJECT_ID,HADM_ID,ADMISSION_DAYS,GENDER,AGE,50801,50802,50803,50804,50805,...,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20
0,22,165315,2,0,65,NaN,0.0,NaN,22.0,NaN,...,0,0,0,0,0,0,0,1,1,0
1,23,152223,6,1,71,NaN,0.0,NaN,26.0,NaN,...,0,0,0,0,0,0,0,0,0,1
2,23,124321,7,1,75,NaN,1.0,NaN,27.0,NaN,...,0,0,0,0,0,1,0,0,0,1
3,24,161859,3,1,39,NaN,0.0,NaN,27.0,NaN,...,0,0,0,0,0,0,0,0,0,0
4,25,129635,4,1,59,NaN,0.0,NaN,26.0,NaN,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58971,98800,191113,3,0,20,230.0,-2.0,26.0,25.0,NaN,...,0,0,0,0,0,0,0,1,1,0
58972,98802,101071,1,0,84,485.0,-3.0,18.0,18.0,NaN,...,0,0,0,0,0,0,0,0,0,1
58973,98805,122631,9,1,42,422.0,-1.0,NaN,27.0,NaN,...,0,1,0,1,0,0,0,1,1,1
58974,98813,170407,11,0,60,NaN,-3.0,22.0,23.0,2.0,...,0,0,0,0,0,0,0,1,1,0


In [20]:

icu_stays = icu_stays.drop(['ROW_ID','DBSOURCE','FIRST_WARDID','LAST_WARDID','INTIME','OUTTIME'],axis=1)
icu_stays.iloc[:,[3]] = lbl.fit_transform(icu_stays.iloc[:,[3]])
icu_stays.iloc[:,[4]] = lbl.fit_transform(icu_stays.iloc[:,[4]])
#MICU-2
#CCU-0
#NICU-3
#TSICU-5
#SICU-4
#CSRU-1
diag_ref = diag_ref.drop(['ROW_ID'],axis=1)

'''lab_itms = lab_itms.drop(['ROW_ID'],axis=1)
lab_itms['CATEGORY'] = lab_itms['CATEGORY'].map(lambda x: x.lower())
lab_itms['CATEGORY'] = lbl.fit_transform(lab_itms['CATEGORY'])
#hematology-2
#blood gas-0
#chemistry-1
lab_itms['FLUID'] = lab_itms['FLUID'].map(lambda x: x.lower())
lab_itms['FLUID'] = lbl.fit_transform(lab_itms['FLUID']) 
#csf -3
#joint fluid-4
#other -5
#pleural-6
#stool=7
#urine-8
#blood-1
#ascites-0
#bone marrow-2'''


/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/sklearn/preprocessing/label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


"lab_itms = lab_itms.drop(['ROW_ID'],axis=1)\nlab_itms['CATEGORY'] = lab_itms['CATEGORY'].map(lambda x: x.lower())\nlab_itms['CATEGORY'] = lbl.fit_transform(lab_itms['CATEGORY'])\n#hematology-2\n#blood gas-0\n#chemistry-1\nlab_itms['FLUID'] = lab_itms['FLUID'].map(lambda x: x.lower())\nlab_itms['FLUID'] = lbl.fit_transform(lab_itms['FLUID']) \n#csf -3\n#joint fluid-4\n#other -5\n#pleural-6\n#stool=7\n#urine-8\n#blood-1\n#ascites-0\n#bone marrow-2"

In [21]:
dataset.shape

(58976, 505)

In [22]:
dataset = dataset.fillna(dataset.median())

In [23]:
dataset = dataset.drop(['SUBJECT_ID','HADM_ID'],axis=1)

In [24]:
dataset

,ADMISSION_DAYS,GENDER,AGE,50801,50802,50803,50804,50805,50806,50808,...,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20
0,2,0,65,462.0,0.0,24.0,22.0,1.0,104.0,1.12,...,0,0,0,0,0,0,0,1,1,0
1,6,1,71,462.0,0.0,24.0,26.0,1.0,104.0,1.20,...,0,0,0,0,0,0,0,0,0,1
2,7,1,75,462.0,1.0,24.0,27.0,1.0,98.0,1.07,...,0,0,0,0,0,1,0,0,0,1
3,3,1,39,462.0,0.0,24.0,27.0,1.0,104.0,1.12,...,0,0,0,0,0,0,0,0,0,0
4,4,1,59,462.0,0.0,24.0,26.0,1.0,104.0,1.11,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58971,3,0,20,230.0,-2.0,26.0,25.0,1.0,101.0,1.12,...,0,0,0,0,0,0,0,1,1,0
58972,1,0,84,485.0,-3.0,18.0,18.0,1.0,104.0,1.12,...,0,0,0,0,0,0,0,0,0,1
58973,9,1,42,422.0,-1.0,24.0,27.0,1.0,103.0,1.08,...,0,1,0,1,0,0,0,1,1,1
58974,11,0,60,462.0,-3.0,22.0,23.0,2.0,111.0,1.00,...,0,0,0,0,0,0,0,1,1,0


### Write to pickle

In [25]:
dataset.to_pickle("./dataset.pkl")

### trying models

In [26]:
np.take(dataset,np.random.permutation(dataset.shape[0]),axis=0)

,ADMISSION_DAYS,GENDER,AGE,50801,50802,50803,50804,50805,50806,50808,...,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20
29417,4,1,81,462.0,0.0,24.0,26.0,1.0,104.0,1.12,...,0,0,0,0,0,0,0,0,0,1
43154,24,0,73,462.0,-2.0,23.0,22.0,1.0,104.0,1.17,...,0,1,0,0,0,0,0,1,1,1
45880,20,0,75,462.0,-6.0,24.0,19.0,1.0,104.0,1.12,...,0,1,1,0,0,1,1,1,0,1
23866,9,0,77,462.0,3.0,24.0,32.0,1.0,104.0,1.12,...,0,1,0,0,0,1,0,0,0,0
17364,11,1,83,462.0,0.0,24.0,26.0,1.0,104.0,1.12,...,0,0,1,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23423,11,0,50,462.0,0.0,24.0,23.0,1.0,104.0,1.08,...,0,0,1,0,0,1,0,0,0,0
666,5,1,77,246.0,2.0,24.0,26.0,1.0,104.0,1.16,...,0,0,0,0,0,1,0,1,1,0
15506,1,1,64,462.0,0.0,24.0,26.0,1.0,104.0,1.12,...,0,0,0,0,0,0,0,0,0,0
27481,12,1,50,185.0,2.0,24.0,26.0,1.0,109.0,1.08,...,0,1,0,0,0,0,0,0,0,0


In [27]:
dataset.columns


Index(['ADMISSION_DAYS',         'GENDER',            'AGE',            50801,
                  50802,            50803,            50804,            50805,
                  50806,            50808,
       ...
                  'G11',            'G12',            'G13',            'G14',
                  'G15',            'G16',            'G17',            'G18',
                  'G19',            'G20'],
      dtype='object', length=503)

In [28]:
dataset.isna().any()

ADMISSION_DAYS    False
GENDER            False
AGE               False
50801             False
50802             False
                  ...  
G16               False
G17               False
G18               False
G19               False
G20               False
Length: 503, dtype: bool

In [29]:
x = dataset.iloc[:,:483]
y = dataset.iloc[:,483:]

In [30]:
x.shape

(58976, 483)

In [31]:
x

,ADMISSION_DAYS,GENDER,AGE,50801,50802,50803,50804,50805,50806,50808,...,51512,51513,51514,51515,51516,51517,51518,51519,51529,51532
0,2,0,65,462.0,0.0,24.0,22.0,1.0,104.0,1.12,...,1.0,1.044,0.2,1.0,0.0,3.0,0.0,0.0,135.5,43.0
1,6,1,71,462.0,0.0,24.0,26.0,1.0,104.0,1.20,...,1.0,1.044,1.0,1.0,2.0,3.0,0.0,0.0,135.5,43.0
2,7,1,75,462.0,1.0,24.0,27.0,1.0,98.0,1.07,...,1.0,1.044,1.0,1.0,2.0,3.0,0.0,0.0,135.5,43.0
3,3,1,39,462.0,0.0,24.0,27.0,1.0,104.0,1.12,...,1.0,1.044,1.0,1.0,2.0,3.0,0.0,0.0,135.5,43.0
4,4,1,59,462.0,0.0,24.0,26.0,1.0,104.0,1.11,...,1.0,1.044,1.0,1.0,2.0,3.0,0.0,0.0,135.5,43.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58971,3,0,20,230.0,-2.0,26.0,25.0,1.0,101.0,1.12,...,1.0,1.044,1.0,1.0,2.0,3.0,0.0,0.0,135.5,43.0
58972,1,0,84,485.0,-3.0,18.0,18.0,1.0,104.0,1.12,...,1.0,1.044,1.0,1.0,2.0,3.0,0.0,0.0,135.5,43.0
58973,9,1,42,422.0,-1.0,24.0,27.0,1.0,103.0,1.08,...,1.0,1.044,1.0,1.0,1.0,3.0,0.0,0.0,135.5,43.0
58974,11,0,60,462.0,-3.0,22.0,23.0,2.0,111.0,1.00,...,1.0,1.044,12.0,1.0,2.0,3.0,0.0,0.0,135.5,43.0


In [32]:
y

,G1,G2,G3,G4,G5,G6,G7,G8,G9,G10,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20
0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0
1,0,0,1,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,1
2,0,1,1,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,1
3,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58971,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0
58972,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
58973,0,1,1,0,0,0,0,1,1,1,0,1,0,1,0,0,0,1,1,1
58974,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0


In [33]:
y.shape

(58976, 20)

In [34]:
x_train = x.iloc[:44232,:].values
y_train = y.iloc[:44232,:].values


In [35]:
x_train.shape

(44232, 483)

In [36]:
#from sklearn.model_selection import train_test_split
#x_train,y_train,x_test,y_test = train_test_split(x.values,y.values,test_size=0.45)

In [37]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/akshara/anaconda3/envs/mimic/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/akshara/anaconda3/envs/mimic/lib/python3

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7967193174095278249
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 11993877725299129526
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 22649336628
locality {
  bus_id: 1
  links {
  }
}
incarnation: 12583517493189003065
physical_device_desc: "device: 0, name: Tesla M40 24GB, pci bus id: 0000:04:00.0, compute capability: 5.2"
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 22649336628
locality {
  bus_id: 2
  numa_node: 1
  links {
  }
}
incarnation: 16669493362115719689
physical_device_desc: "device: 1, name: Tesla M40 24GB, pci bus id: 0000:82:00.0, compute capability: 5.2"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 4056689003348740319
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_GPU:1"
device

In [38]:
import keras
from keras.layers import Dense,CuDNNLSTM,CuDNNGRU,Flatten,Conv1D,Dropout
from keras.models import Sequential

Using TensorFlow backend.


In [39]:
x_train.shape

(44232, 483)

In [40]:
y_train.shape

(44232, 20)

In [41]:
x_train = np.reshape(x_train,(x_train.shape[0],1,x_train.shape[1]))
y_train = np.reshape(y_train,(y_train.shape[0],1,y_train.shape[1]))

In [42]:
x_train.shape


(44232, 1, 483)

In [43]:
y_train.shape

(44232, 1, 20)

In [44]:

model = Sequential()
model.add(CuDNNLSTM(256,input_shape=(x_train.shape[1],x_train.shape[2])))
model.add(CuDNNGRU(256))
model.add(CuDNNLSTM(256))
model.add(CuDNNGRU(256))
model.add(Dense(1,activation='sigmoid'))


ValueError: Input 0 is incompatible with layer cu_dnngru_1: expected ndim=3, found ndim=2

In [ ]:
'''input_shape = x_train.shape[0]
model = Sequential()
model.add(Conv1D(filters=256,kernel_size=1,input_shape=(x_train.shape[1],x_train.shape[2])))
#model.add(Dense(units=256, activation='relu'))
model.add(Dense(units=512, activation='relu'))
model.add(Dropout(0.3))
model.add(Conv1D(filters=128,kernel_size=1))
model.add(Dense(units=1024,activation='relu'))
#model.add(Dense(units=2048,activation='relu'))
model.add(Dropout(0.3))
model.add(Conv1D(filters=128,kernel_size=1))
model.add(Dense(units=4096,activation='relu'))
model.add(Dense(y_train.shape[2],activation='sigmoid'))'''

In [ ]:
model.compile(metrics=['accuracy'],loss='binary_crossentropy',optimizer = 'adam')

In [ ]:
model.summary()

In [ ]:
#from skmultilearn.problem_transform import LabelPowerset
#classifier = LabelPowerset()

In [ ]:
model.fit(x_train,y_train,epochs=20,batch_size=128,validation_split=0.3)


In [ ]:
print(model.summary())

In [ ]:
x_test = x.iloc[44233:,:].values
y_test = y.iloc[44233:,:].values
x_test = np.reshape(x_test,(x_test.shape[0],1,x_test.shape[1]))


In [ ]:
x_test.shape

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
y_pred.shape

In [ ]:
y_pred = np.reshape(y_pred,(y_pred.shape[0],y_pred.shape[2]))

In [ ]:
y_test.shape


In [ ]:
y_pred

In [ ]:
y_pred

In [ ]:
y_test

In [ ]:
from sklearn.metrics import jaccard_score,roc_auc_score
#print(confusion_matrix(y_test,y_pred))
print(jaccard_score(y_test,y_pred,average='macro'))
print(jaccard_score(y_test,y_pred,average='micro'))
print(jaccard_score(y_test,y_pred,average='weighted'))
#print(roc_auc_score(y_test,y_pred,average='macro'))
print(roc_auc_score(y_test,y_pred,average='micro'))
#print(roc_auc_score(y_test,y_pred,average='weighted'))

In [ ]:
from sklearn.metrics import hamming_loss
print(hamming_loss(y_test,y_pred))

In [ ]:
dataset.shape[0]*dataset.shape[1]